In [24]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

data1 = pd.read_csv("../data_after_cleaning.csv")

SEED = 4

X_without_noise = data1.drop(columns=['type'])
y_without_noise = data1['type']

X_train_without_noise, X_test_without_noise, y_train_without_noise, y_test_without_noise = train_test_split(X_without_noise, y_without_noise, test_size=0.4, random_state=SEED)


In [25]:
from sklearn.linear_model import Lasso
import numpy as np

# lasso 回归 模型

# 模型参数
ALPHA = 0.0125

# 学习模型
lasso = Lasso(alpha=ALPHA)
lasso.fit(X_train_without_noise, y_train_without_noise)

def lasso_classification_func(X):
    return np.where(lasso.predict(X) >= 1 / 2, 1, 0)


In [26]:
# XGBOOST 决策树模型

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb=XGBClassifier(max_depth=2)
xgb.fit(X_train_without_noise,y_train_without_noise)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [27]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

std = StandardScaler()

pca = PCA(n_components=2)
data_train = X_train_without_noise
color = y_train_without_noise
data_after_pca = pca.fit_transform(data_train)
data_after_pca = pd.DataFrame(data_after_pca)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(data_after_pca, color)

def pca_predict(X):
    X = pca.transform(X)
    return knn.predict(pd.DataFrame(X))


In [28]:
import pandas as pd

degree_dict = {
    "无风化" : 0,
    "风化"   : 1,
}

data = pd.read_csv("../sheet2_origin.csv").fillna(0)
data['degree'] = data['degree'].map(degree_dict)

X_test = data[X_without_noise.columns]

y_pred_lasso = lasso_classification_func(X_test)
y_pred_xgb = xgb.predict(X_test)
y_pred_pca = pca_predict(X_test)

print(f'lasso: {y_pred_lasso}\nxgboost: {y_pred_xgb}\npca+knn:{y_pred_pca}')

lasso: [1 0 0 0 0 1 1 0]
xgboost: [1 0 0 0 0 1 1 0]
pca+knn:[1 0 0 0 0 1 1 0]
